In [14]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import *
from skimage.metrics import structural_similarity as ssim

In [18]:
!rm -rf /content/keras-ocr/

In [4]:
!git clone https://github.com/faustomorales/keras-ocr.git

Cloning into 'keras-ocr'...
remote: Enumerating objects: 1044, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 1044 (delta 11), reused 20 (delta 7), pack-reused 1010
Receiving objects: 100% (1044/1044), 1.06 MiB | 8.31 MiB/s, done.
Resolving deltas: 100% (657/657), done.


In [30]:
%cd /content

/content


In [ ]:
!pip install efficientnet

In [ ]:
!pip install validators

In [31]:
def imshow(im):
    width, height, *channels = im.shape
    plt.figure(figsize=(10,10))
    if channels:
        # By default, OpenCV tends to work with images in the BGR format.
        # This is due to some outdated practices, but it has been left in the library.
        # We can iterate the channels in reverse order to get an RGB image.
        plt.imshow(im[:,:,::-1])
    else:
        plt.imshow(im, cmap='gray')
    plt.axis('off')

In [16]:
def preprocessing(image_path):
  im = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
  im_rgb = cv2.imread(image_path)
  edited= cv2.GaussianBlur(im,(1,1),3)
  # plt.imshow(edited, cmap='gray')
  # plt.show()

  th3 = cv2.adaptiveThreshold(edited ,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,1001,0)
  edgeimage= cv2.Canny(th3,30,50)
  # plt.imshow(edgeimage,cmap="gray")
  # plt.show()
  contours,_=cv2.findContours(edgeimage,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
  max_con = contours[0]
  max_con_A = len(max_con)
  for cnt in contours:
      if len(cnt) > max_con_A:
          max_con_A = len(cnt)
          max_con = cnt
  res = im_rgb.copy()
  epsilon = 0.01 * cv2.arcLength(
                              max_con, True)
  approx = cv2.approxPolyDP(max_con, epsilon, True)

  pts1 = np.float32([approx[0], approx[1], approx[2], approx[3]])
  width, height = im.shape[0], im.shape[1]
  pts2 = np.float32([[width, 0], [0, 0], [0, height], [width, height]])
  M = cv2.getPerspectiveTransform(pts1, pts2)
  output_img = cv2.warpPerspective(res, M, (width, height))
  return output_img

In [17]:
def maching(img, template):

    mach = 0

    rotate_image=[]
    rotate_image.append(img)
    rotate_image.append(  cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE))
    rotate_image.append(   cv2.rotate(img, cv2.ROTATE_180))
    rotate_image.append(   cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE))
    for rotated in rotate_image:


        # Perform template matching
        temp = cv2.resize(template, (136, 123))
        image = cv2.resize(rotated, (1280, 827))
        result = cv2.matchTemplate(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), cv2.cvtColor(temp, cv2.COLOR_BGR2GRAY), cv2.TM_CCOEFF_NORMED)

        if np.max(result) > 0.65:
            mach = 1
            break



    if mach == 1:
        return True

    # Perform template matching with flipped template
    result_flipped = cv2.matchTemplate(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), cv2.cvtColor(template, cv2.COLOR_BGR2GRAY), cv2.TM_CCOEFF_NORMED)
    if np.max(result_flipped) > 0.65:
        return True

    return False


In [18]:
def get_points_of_rectangle(potentials, rects): # potentials items : (content, index, area)
  most_left_top = sorted(potentials, key= lambda x: rects[x[1]][1][0][0])[0]
  most_right_bottom = sorted(potentials, key= lambda x: rects[x[1]][1][2][0])[-1]
  return rects[most_left_top[1]][1][0], rects[most_right_bottom[1]][1][2]

In [19]:
def find_number(potentials, rects):
  nums = sorted(potentials, key= lambda x: rects[x[1]][1][0][0])
  return nums[0][0]+nums[1][0]+nums[2][0]+nums[3][0]

In [20]:
def detect_info(images, prediction_groups, is_identity):
  info = []
  for i in range(len(prediction_groups)):
    if is_identity[i]:
      info.append("it is a identity card")
      continue
    im = images[i]
    potentials = []
    for ind,pred in enumerate(prediction_groups[i]):
      top_left,top_right,bottom_right,bottom_left = (pred[1][0],pred[1][1],pred[1][2],pred[1][3])
      wid = (top_left[0]-top_right[0])**2 + (top_left[1]-top_right[1])**2
      height = (bottom_left[0]-bottom_right[0])**2 + (bottom_left[1]-bottom_right[1])**2
      area = wid*height
      if pred[0].isdigit() and len(pred[0])==4:
        potentials.append((pred[0],ind,area))
    potentials = sorted(potentials, key=lambda x: x[2], reverse=True)
    card_number_parts = potentials[:4]
    card_num = find_number(card_number_parts, prediction_groups[i])
    pt1,pt2 = get_points_of_rectangle(card_number_parts, prediction_groups[i])
    s,t = (int(pt1[0]),int(pt1[1])), (int(pt2[0]),int(pt2[1]))
    info.append((card_num, s, t))
  return info

In [22]:
import matplotlib.pyplot as plt
import os
import shutil
from google.colab.patches import cv2_imshow
# keras-ocr will automatically download pretrained
# weights for the detector and recognizer.
def predict(images):
  if os.path.exists("images"):
    shutil.rmtree("images")

  # Create new images directory
  os.makedirs("images")
  template = cv2.imread("/content/iran2.jpg")
  is_identity = [False]*len(images)
  pre_images = []
  for i,im in enumerate(images):
    preprocessed_img = preprocessing(im)
    # Save output image
    output_path = os.path.join("images", f"{i}_processed.jpg")
    cv2.imwrite(output_path, preprocessed_img)
    pre_im = cv2.imread(f"/content/images/{i}_processed.jpg")
    pre_images.append(pre_im)
    is_identity[i] = maching(pre_im, template)

  %cd keras-ocr/keras_ocr/
  import pipeline, tools
  pipeline = pipeline.Pipeline()
  # Get a set of three example images
  urls = []
  for u in range(len(images)):
    urls.append(f"/content/images/{u}_processed.jpg")
  images = [
      tools.read(url) for url in urls
  ]

  # Each list of predictions in prediction_groups is a list of
  # (word, box) tuples.
  prediction_groups = pipeline.recognize(images)
  fig, axs = plt.subplots(nrows=len(images), figsize=(20, 20))
  for ax, image, predictions in zip(axs, images, prediction_groups):
    tools.drawAnnotations(image=image, predictions=predictions, ax=ax)
  info = detect_info(images, prediction_groups, is_identity)
  for i in range(len(images)):
    if isinstance(info[i], str):
      print(info[i])
      cv2_imshow(images[i])
    else:
      print(info[i][0])
      im = cv2.rectangle(im, info[i][1], info[i][2], (0, 0, 255), 2)
      cv2_imshow(im)

  # Plot the predictions


In [ ]:
predict(["/content/3.jpg","/content/8.jpg"])

/content/keras-ocr/keras_ocr
Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5
